In [1]:
import pandas as pd

RANDOM_STATE=137

In [2]:
dtype = {
    'cough': 'float32',
    'fever': 'float32',
    'sore_throat': 'float32',
    'shortness_of_breath': 'float32',
    'head_ache': 'float32',
    'corona_result': 'str',
    'age_60_and_above': 'str',
    'gender': 'str',
    'test_indication': 'str'
}
covid = pd.read_csv('data/corona_dataset_preprocessed.csv')
covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274702 entries, 0 to 274701
Data columns (total 8 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   test_date              274702 non-null  object
 1   cough                  274702 non-null  int64 
 2   fever                  274702 non-null  int64 
 3   sore_throat            274702 non-null  int64 
 4   shortness_of_breath    274702 non-null  int64 
 5   head_ache              274702 non-null  int64 
 6   test_indication        274702 non-null  object
 7   encoded_corona_result  274702 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 16.8+ MB


In [11]:
positive = covid.loc[covid.encoded_corona_result == 1]
positive.head()

,test_date,cough,fever,sore_throat,shortness_of_breath,head_ache,test_indication,encoded_corona_result
41,2020-04-30,1,0,0,0,0,Contact with confirmed,1
43,2020-04-30,1,0,0,0,0,Other,1
102,2020-04-30,0,0,0,0,0,Contact with confirmed,1
119,2020-04-30,0,0,0,0,0,Contact with confirmed,1
136,2020-04-30,1,1,0,0,0,Other,1


In [12]:
positive.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14694 entries, 41 to 274700
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   test_date              14694 non-null  object
 1   cough                  14694 non-null  int64 
 2   fever                  14694 non-null  int64 
 3   sore_throat            14694 non-null  int64 
 4   shortness_of_breath    14694 non-null  int64 
 5   head_ache              14694 non-null  int64 
 6   test_indication        14694 non-null  object
 7   encoded_corona_result  14694 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 1.0+ MB


In [8]:
from sklearn.pipeline import Pipeline

from sklearn.impute import SimpleImputer

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler

from sklearn.compose import ColumnTransformer

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split


# Models
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier

# Evaluation Metric
from sklearn.metrics import roc_auc_score

num_features = ['cough', 'fever', 'sore_throat', 'shortness_of_breath', 'head_ache']
cat_features = [
    'test_indication', 
    # 'age_60_and_above', 
    # 'gender'
]

num_transformer = Pipeline([
    # ('imputer', SimpleImputer(strategy='median'))
])

cat_transformer = Pipeline([
    ('encoder', OneHotEncoder()),
    # ("selector", SelectPercentile(chi2, percentile=50))
])

preprocessor = ColumnTransformer(
    transformers=[
        # ('num', num_transformer, num_features),
        ('cat', cat_transformer, cat_features)
    ],
    remainder='passthrough'
)

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    # ('scaler', 'passthrough'),
    #('classifier', LogisticRegression(solver='liblinear')),
    ('classifier', XGBClassifier())
    # ('classifier', DecisionTreeClassifier())
])

params = {
    # 'scaler': [StandardScaler(), MaxAbsScaler()],
    # 'classifier__learning_rate': [0.3, 0.5, 0.7],
    'classifier__n_estimators': [
        10, 
        50, 
        100,
        150,
        200
    ],
    'classifier__max_depth': [
        2, 
        4, 
        6,
        8,
        10
    ]
}

log_reg_params = {
    'classifier__C': [
        0.01,
        0.1,
        1,
        10,
        100
    ],
    'classifier__penalty': [
        'l1',
        'l2'
    ]
}


model = GridSearchCV(
    estimator=pipeline,
    param_grid=log_reg_params,
    scoring='roc_auc', # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    cv=2
)

X = covid.drop(columns=['encoded_corona_result', 'test_date']).copy()
y = covid.encoded_corona_result.copy()


In [27]:
X.head()

,cough,fever,sore_throat,shortness_of_breath,head_ache,test_indication
0,0,0,0,0,0,Other
1,1,0,0,0,0,Other
2,0,1,0,0,0,Other
3,1,0,0,0,0,Other
4,1,0,0,0,0,Other


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=RANDOM_STATE)

# model = XGBClassifier(objective='multi:softmax')
# model = DecisionTreeClassifier()
model.fit(X_train, y_train)

/home/matt/apprenticeship-hackathon/.python/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [09:14:01] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "C", "penalty" } are not used.

  warnings.warn(smsg, UserWarning)
/home/matt/apprenticeship-hackathon/.python/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [09:14:04] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "C", "penalty" } are not used.

  warnings.warn(smsg, UserWarning)
/home/matt/apprenticeship-hackathon/.python/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [09:14:05] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "C", "penalty" } are not used.

  warnings.warn(smsg, UserWarning)
/home/matt/apprenticeship-hackathon/.python/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [09:14:07] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "C", "penalty" } are not used.

  warnings.warn(smsg, UserWarning)
/home/matt/apprenticeship-hackathon/.pyt

GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('cat',
                                                                         Pipeline(steps=[('encoder',
                                                                                          OneHotEncoder())]),
                                                                         ['test_indication'])])),
                                       ('classifier',
                                        XGBClassifier(base_score=None,
                                                      booster=None,
                                                      callbacks=None,
                                                      colsample_bylevel=None,
                                                      colsample_bynode=None,
                                                      colsample_bytree=None,
                                                      device=None,
                                                      early_...
                                                      max_cat_threshold=None,
                                                      max_cat_to_onehot=None,
                                                      max_delta_step=None,
                                                      max_depth=None,
                                                      max_leaves=None,
                                                      min_child_weight=None,
                                                      missing=nan,
                                                      monotone_constraints=None,
                                                      multi_strategy=None,
                                                      n_estimators=None,
                                                      n_jobs=None,
                                                      num_parallel_tree=None,
                                                      random_state=None, ...))]),
             param_grid={'classifier__C': [0.01, 0.1, 1, 10, 100],
                         'classifier__penalty': ['l1', 'l2']},
             scoring='roc_auc')

In [10]:
predicted_probabilities = model.predict_proba(X_test)
roc_auc_score(y_test, predicted_probabilities[:, -1])*100

89.09360560752114

In [19]:
data = []
for cough in range(0,2):
    for fever in range(0,2):
        for sore_throat in range(0,2):
            for shortness_of_breath in range(0,2):
                for head_ache in range(0,2):
                    for contact in range(0,3):
                        contact_str = 'Other'
                        if contact == 1:
                            contact_str = 'Contact with confirmed'
                        elif contact == 2:
                            contact_str = 'Abroad'
                        data.append([cough, fever, sore_throat, shortness_of_breath, head_ache, contact_str])
print(data)

[[0, 0, 0, 0, 0, 'Other'], [0, 0, 0, 0, 0, 'Contact with confirmed'], [0, 0, 0, 0, 0, 'Abroad'], [0, 0, 0, 0, 1, 'Other'], [0, 0, 0, 0, 1, 'Contact with confirmed'], [0, 0, 0, 0, 1, 'Abroad'], [0, 0, 0, 1, 0, 'Other'], [0, 0, 0, 1, 0, 'Contact with confirmed'], [0, 0, 0, 1, 0, 'Abroad'], [0, 0, 0, 1, 1, 'Other'], [0, 0, 0, 1, 1, 'Contact with confirmed'], [0, 0, 0, 1, 1, 'Abroad'], [0, 0, 1, 0, 0, 'Other'], [0, 0, 1, 0, 0, 'Contact with confirmed'], [0, 0, 1, 0, 0, 'Abroad'], [0, 0, 1, 0, 1, 'Other'], [0, 0, 1, 0, 1, 'Contact with confirmed'], [0, 0, 1, 0, 1, 'Abroad'], [0, 0, 1, 1, 0, 'Other'], [0, 0, 1, 1, 0, 'Contact with confirmed'], [0, 0, 1, 1, 0, 'Abroad'], [0, 0, 1, 1, 1, 'Other'], [0, 0, 1, 1, 1, 'Contact with confirmed'], [0, 0, 1, 1, 1, 'Abroad'], [0, 1, 0, 0, 0, 'Other'], [0, 1, 0, 0, 0, 'Contact with confirmed'], [0, 1, 0, 0, 0, 'Abroad'], [0, 1, 0, 0, 1, 'Other'], [0, 1, 0, 0, 1, 'Contact with confirmed'], [0, 1, 0, 0, 1, 'Abroad'], [0, 1, 0, 1, 0, 'Other'], [0, 1, 0, 1, 

In [23]:
len(data)

96

In [28]:
import numpy as np
dataframe = pd.DataFrame(data, columns=['cough','fever','sore_throat','shortness_of_breath','head_ache','test_indication'])
dataframe.head()

,cough,fever,sore_throat,shortness_of_breath,head_ache,test_indication
0,0,0,0,0,0,Other
1,0,0,0,0,0,Contact with confirmed
2,0,0,0,0,0,Abroad
3,0,0,0,0,1,Other
4,0,0,0,0,1,Contact with confirmed


In [29]:
myPred = model.predict(dataframe)

In [30]:
print(myPred)

[0 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1
 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [31]:
results = dataframe
results['covid_pred'] = myPred
results.head()

,cough,fever,sore_throat,shortness_of_breath,head_ache,test_indication,covid_pred
0,0,0,0,0,0,Other,0
1,0,0,0,0,0,Contact with confirmed,1
2,0,0,0,0,0,Abroad,0
3,0,0,0,0,1,Other,1
4,0,0,0,0,1,Contact with confirmed,1


In [32]:
print(results)

    cough  fever  sore_throat  shortness_of_breath  head_ache  \
0       0      0            0                    0          0   
1       0      0            0                    0          0   
2       0      0            0                    0          0   
3       0      0            0                    0          1   
4       0      0            0                    0          1   
..    ...    ...          ...                  ...        ...   
91      1      1            1                    1          0   
92      1      1            1                    1          0   
93      1      1            1                    1          1   
94      1      1            1                    1          1   
95      1      1            1                    1          1   

           test_indication  covid_pred  
0                    Other           0  
1   Contact with confirmed           1  
2                   Abroad           0  
3                    Other           1  
4   Contact wi

In [33]:
with pd.option_context('display.max_rows', None):
    display(results)

,cough,fever,sore_throat,shortness_of_breath,head_ache,test_indication,covid_pred
0,0,0,0,0,0,Other,0
1,0,0,0,0,0,Contact with confirmed,1
2,0,0,0,0,0,Abroad,0
3,0,0,0,0,1,Other,1
4,0,0,0,0,1,Contact with confirmed,1
5,0,0,0,0,1,Abroad,1
6,0,0,0,1,0,Other,0
7,0,0,0,1,0,Contact with confirmed,1
8,0,0,0,1,0,Abroad,1
9,0,0,0,1,1,Other,1


In [34]:
negative = results.loc[results.covid_pred == 0]
with pd.option_context('display.max_rows', None):
    display(negative)

,cough,fever,sore_throat,shortness_of_breath,head_ache,test_indication,covid_pred
0,0,0,0,0,0,Other,0
2,0,0,0,0,0,Abroad,0
6,0,0,0,1,0,Other,0
24,0,1,0,0,0,Other,0
26,0,1,0,0,0,Abroad,0
48,1,0,0,0,0,Other,0
49,1,0,0,0,0,Contact with confirmed,0
50,1,0,0,0,0,Abroad,0
56,1,0,0,1,0,Abroad,0
62,1,0,1,0,0,Abroad,0
